In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [ ]:
df_train = pd.read_csv("../input/home-data-for-ml-course/train.csv")
df_test = pd.read_csv("../input/home-data-for-ml-course/train.csv")
df_full = pd.concat([df_train,df_test]).reset_index(drop = True)           
sale_price = df_train['SalePrice'].reset_index(drop=True)
df_full.drop('SalePrice',axis=1,inplace=True)

##### 

In [ ]:
df_full.shape
#print(df_train.shape)
#df_train.describe()

STEP 1 : Let's check the column and the missing values

In [ ]:
col_with_missing = df_full.columns[df_full.isna().sum()>0]
col_with_missing
#col_with_missing = df_train.columns[df_train.isna().sum()>0]
#col_with_missing

In [ ]:
for col in df_full.columns:
    if((df_full[col].isna().sum()/df_full.shape[0])>=0.4):
        df_full.drop(col,axis=1,inplace=True)

df_full.shape

#fill_with_none = ['Alley','PoolQC','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1',
 #                 'BsmtFinType2','FireplaceQu','GarageType','GarageFinish','GarageQual',
  #                'GarageCond','Fence','MiscFeature','MasVnrType']

#df_train[fill_with_none] = df_train[fill_with_none].fillna('None')
#df_train.GarageYrBlt = df_train['GarageYrBlt'].fillna(0).astype('int')
#df_train.MasVnrArea = df_train.MasVnrArea.fillna(0).astype('int')
#df_train.drop('Electrical', axis=1, inplace=True)
#df_train.LotFrontage = df_train.LotFrontage.fillna(df_train.LotFrontage.median()).astype('int')

In [ ]:
cat_cols = df_full.select_dtypes(include='object')
num_cols = df_full.select_dtypes(exclude='object')
scaler = MinMaxScaler()

for col in num_cols:
    df_full[col] = df_full[col].fillna(df_full[col].median())
    scaler.fit_transform(np.array(df_full[col]).reshape(-1,1))

#check if there is still na values
#print(df_train.columns[df_train.isna().sum()>0] ,'\n'*2)

#cat_cols = df_train.select_dtypes(include='object').columns.tolist()
#num_cols = df_train.select_dtypes(exclude='object').columns.tolist()
#print("categorical cols : {} '\n' numerical cols : {}".format(cat_cols, num_cols))

In [ ]:
df_full.head()
#for col in cat_cols:
#    print(col , df_train[col].unique())

In [ ]:
#cat_to_num = {'None':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
#list_cat_to_num = ['BsmtQual','BsmtCond','GarageCond','KitchenQual','PoolQC','HeatingQC',
#                   'ExterCond','ExterQual']
#df_train[list_cat_to_num] = df_train[list_cat_to_num].replace(cat_to_num)

In [ ]:
corr = df_train.corr()
plt.figure(figsize=(19,10))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, vmax=0.7, cmap = cmap, square=True)

sale_price_corr = df_train.corr()['SalePrice'][:-1]
sale_price_corr.sort_values(ascending=False)

In [ ]:
#dropping columns with less than 12% correlation with y 
for column,row in sale_price_corr.iteritems():
    if abs(row)<=0.12:
        df_full.drop(column,axis=1,inplace=True)



#trying to remove all categorical columns that I didn't transform in numerical

#train = df_train.copy()
#cat_cols2 = cat_cols.copy()
#print(list_cat_to_num)
#for col in list_cat_to_num:
#    cat_cols2.remove(col)
#train.drop(cat_cols2,axis=1,inplace=True)

In [ ]:
df_full.mode().iloc[0]

In [ ]:
#removing cartegorical columns with more than 6 differents unique values (too much)
for col in cat_cols:
    if df_full[col].nunique() >=6:
        df_full.drop(col,axis=1,inplace=True)

df_full = df_full.fillna(df_full.mode().iloc[0])
#y = train.SalePrice
#train.drop('SalePrice', axis=1,inplace=True)
#X_train,X_valid,y_train,y_valid = train_test_split(train,y, train_size = 0.8)
#scaler = MinMaxScaler()
#scaler.fit(X_train)
#X_train,X_valid = scaler.transform(X_train), scaler.transform(X_valid)

In [ ]:
le = LabelEncoder()
for col in df_full.columns:
    if df_full[col].dtype=='object' and df_full[col].nunique()>2:
        df_full = pd.concat([df_full,pd.get_dummies(df_full[col], prefix = str(col))],axis=1)
        df_full.drop(col,axis=1,inplace=True)
    elif df_full[col].dtype=='object' and df_full[col].nunique() <=2:
        df_full[col] = le.fit_transform(df_full[col])
        df_full.drop(col,axis=1,inplace=True)

df_full
        
#model = XGBRegressor(learning_rate = 0.1,
#              max_depth = 6,
#              min_child_weight = 10,
#              subsample = 0.7,
#              colsample_bytree= 1,
#              reg_lambda = 1,
#              gamma = 0.005,
#              reg_alpha = 0.0005,
#              n_estimators = 202)
#model.fit(X_train,y_train)
#rmse = mean_squared_error(model.predict(X_valid),y_valid,squared=False)
#print(rmse)

In [ ]:
train_df = df_full[:df_train.shape[0]]
test_df =  df_full[df_train.shape[0]:]

In [ ]:
xgboost = XGBRegressor(learning_rate=0.008,
                       n_estimators=6000,
                       max_depth=4,
                       min_child_weight=0,
                       gamma=0.6,
                       subsample=0.7,
                       colsample_bytree=0.7,
                       objective='reg:squarederror',
                       nthread=-1,
                       scale_pos_weight=1,
                       seed=27,
                       reg_alpha=0.00006,
                       random_state=42)

xgb = xgboost.fit(train_df,sale_price)
training_accuracy = xgb.score(train_df,sale_price)
print("Training accuracy: %.2f%%" % (training_accuracy * 100.0))
xgb_predictions = xgb.predict(test_df)

In [ ]:
submission = pd.DataFrame({'ID':df_test['Id'],'SalePrice':xgb_predictions})
submission.to_csv('housepricepred3.csv',index = False)
submission